In [1]:
import sys  # For running system level commands
import yaml  # For reading the config file
import os  # For path tools
import requests  # For symphony client
import symphony_client  # For connecting to Symphony region
from pip._vendor.distlib.compat import raw_input
import datetime
from pytz import timezone
import pytz  # To add timezone to datetime

In [2]:
!{sys.executable} -m pip install pyyaml

DEPRECATION: Python 2.7 will reach the end of its life on January 1st, 2020. Please upgrade your Python as Python 2.7 won't be maintained after that date. A future version of pip will drop support for Python 2.7. More details about Python 2 support in pip, can be found at https://pip.pypa.io/en/latest/development/release-process/#python-2-support
You should consider upgrading via the 'pip install --upgrade pip' command.


In [3]:
print(u"[INIT] Initialising script.")
# Configure environment
tz_utc = pytz.timezone("UTC") # Set timezone for data source
rootpath = os.path.dirname(os.path.realpath('__file__'))  # Get the root path

[INIT] Initialising script.


In [4]:
# Import config file data
with open(rootpath + '/config.yml', 'r') as stream:
    try:
        config = yaml.safe_load(stream)
        print(u" [\u2713] Config file loaded.")
    except yaml.YAMLError as exc:
        print(u" [\u2717] Could not load the config file.")
        print(exc)
        exit()
class textCol:
    RED = '\033[91m'
    GREEN = '\033[92m'
    YELLOW = '\033[93m'
    BLUE = '\033[94m'
    PURPLE = '\033[95m'
    CYCAN = '\033[96m'
    END = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

 [✓] Config file loaded.


In [5]:
# Configure Stratoscale API connection
symp_url = "https://" + config["region_access"]["ipaddress"]
symp_domain = config["region_access"]["cloud_domain"]
symp_user = config["region_access"]["cloud_user"]
symp_password = config["region_access"]["cloud_password"]
symp_project = config["region_access"]["project"],
my_session = requests.Session()
my_session.verify = False

try:
    client = symphony_client.Client(url=symp_url, session=my_session)
    client_login = client.login(domain=symp_domain, username=symp_user, password=symp_password,project=symp_project)
    print(u" [\u2713] Stratoscale user region [{}] session established.".format(symp_url))
except:
    print(u" [\u2717] Could not connect to the Stratosacle region [{}] as user".format(symp_url))
    exit()

 [✓] Stratoscale user region [https://192.168.101.100] session established.


In [6]:
dbs = client.dbs.instance.list()
count = 0
for db in dbs:
    count = count + 1
    if db.status.lower() == "active":
        status = "{}[Active]".format(textCol.GREEN)
    elif db.status.lower() == "pending":
        status = "{}[Pending]".format(textCol.PURPLE)
    elif db.status.lower() == "processing":
        status = "{}[Processing]".format(textCol.CYCAN)
    elif db.status.lower() == "stopped":
        status = "{}[Stopped]".format(textCol.BLUE)
    else:
        status = "{}Error".format(textCol.RED)
    db_details = "-------------------------------------------------------------------------------------------"
    db_details = ("{}\n [{}] {}{} [{}]{}\n  DB ID {}\t{}{}".format(
        db_details,
        count,
        textCol.BOLD,
        db.name,
        db.floating_ip,
        textCol.END,
        db.id,
        status,
        textCol.END
    ))
    db_data = client.dbs.instance.get(db.id)
    try:
        percent_used = 100 - int(db_data.stats.get("df/percent_bytes__free"))
        if percent_used > 85:
            percent_used = "{}{}[CRITICAL] - {}".format(textCol.BOLD, textCol.RED, percent_used)
        elif percent_used > 80:
            percent_used = "{}[ERROR] - {}".format(textCol.PURPLE, percent_used)
        elif percent_used > 70:
            percent_used = "{}[WARNING] - {}".format(textCol.YELLOW, percent_used)
        else:
            percent_used = "{}[OK] - {}".format(textCol.GREEN, percent_used)
    except:
        percent_used = "{}[No Data] - ?".format(textCol.YELLOW)
    capacity_details = "  Data Volume\t\t\t\t\t{}% consumed of the allocated {}GB{}".format(
        percent_used,
        db.allocated_storage,
        textCol.END
    )
    print(db_details)
    print(capacity_details)

-------------------------------------------------------------------------------------------
 [1] Happi [172.16.250.97]
  DB ID 05764e6a-cd65-4739-a7ea-f4efa04ee06f	[Active]
  Data Volume					[OK] - 9% consumed of the allocated 14GB
-------------------------------------------------------------------------------------------
 [2] Replica_ignes_db_training [172.16.250.63]
  DB ID 0993287b-8213-402a-ab3a-7eadf2bdb5f7	[Active]
  Data Volume					[OK] - 10% consumed of the allocated 9GB
-------------------------------------------------------------------------------------------
 [3] RemoteReplicaBaas [172.16.250.135]
  DB ID 255647c7-7d84-4c2c-b82f-9f1ffddc0c9e	[Active]
  Data Volume					[No Data] - ?% consumed of the allocated 3GB
-------------------------------------------------------------------------------------------
 [4] MyDB1ca [172.16.250.117]
  DB ID 2da6d6f4-f6bb-470c-b28b-034ff7b1fe34	[Active]
  Data Volume					[OK] - 10% consumed of the allocated 9GB
--------------------------------